## 요리사

In [2]:
def food_s(cnt, subtree, n):
    global ans
    if cnt == n:
        lst1 = subtree[:n//2]
        lst2 = subtree[n//2:]
        food_a = 0
        food_b = 0
        for i in range(n//2):
            for j in range(n//2):
                if i < j:
                    food_a += arr[lst1[i]][lst1[j]] + arr[lst1[j]][lst1[i]]
                    food_b += arr[lst2[i]][lst2[j]] + arr[lst2[j]][lst2[i]]
        sum_v = abs(food_a - food_b)
        if sum_v < ans:
            ans = sum_v
        return
    else:
        for value in food_list:
            if value not in subtree:
                subtree.append(value)
                food_s(cnt+1, subtree, n)
                subtree.pop()
    return
T = int(input())
for tc in range(1, T+1):
    N = int(input())
    arr = [[int(a) for a in input().split()]for _ in range(N)]
    food_list = [int(i) for i in range(N)]
    ans = float('inf')

    food_s(0, [], N)

    print(f'#{tc} {ans}')

- 기존의 코드는 구현은 됐음. 하지만 제한시간 초과.
- 방식: 순열로 가능한 모든 경우의 수 를 구하고 -> 연산 진행
- 문제점: [0,1,2,3]순열과 [3,2,1,0]순열의 경우 어차피 같은 결과값을 가지고 있으나, 연산을 중복으로 하기 때문에 시간복잡도가 높았음. 22/50개 정답.

In [ ]:

def food_s(cnt, idx, selected):
    global ans
    if cnt == N // 2:   # 반, 반 나눠서 selected에 둘 다 있으면 food_a, 둘다 없으면 food_b
        food_a, food_b = 0, 0
        for i in range(N):      
            for j in range(i + 1, N):
                if selected[i] and selected[j]:
                    food_a += arr[i][j] + arr[j][i]
                elif not selected[i] and not selected[j]:
                    food_b += arr[i][j] + arr[j][i]
        ans = min(ans, abs(food_a - food_b))
        return

    for i in range(idx, N):
        selected[i] = True
        food_s(cnt + 1, i + 1, selected)
        selected[i] = False


T = int(input())
for tc in range(1, T + 1):
    N = int(input())
    arr = [list(map(int, input().split())) for _ in range(N)]
    selected = [False] * N  # 방문 여부를 판단. 
    ans = float('inf')  # 최솟값을 저장하는 변수
    food_s(0, 0, selected)  
    print(f'#{tc} {ans}')

- 최종 코드
- selected 라는 리스트를 만들고, 해당 리스트를 for문을 통해서 순회하며 selected의 길이가 N//2가 될때까지 반복해서 arr리스트의 길이의 절반만큼 순회를 하면 if문에 들어가서 음식a와 음식b를 연산.

## 활주로 건설

In [ ]:
def check(lst, v):
    cnt = 1 # 연속된 동일 숫자의 개수 카운트
    for i in range(1, len(lst)):
        if lst[i-1] == lst[i]:
            cnt += 1
            # 이전 높이보다 높아졌다면 -> L길이 만큼의 여유공간이 있으면서 -> 경사로가 설치가 되어있는지 확인
        elif lst[i-1]+1 == lst[i] and cnt >= L and v[i-L:i] == [0]*L:
            v[i - L:i] = [1] * L    # 경사로 놓기
            cnt = 1
        elif lst[i-1] > lst[i]: # 높이가 1칸 낮아지면 cnt = 1로 다시 초기화
            cnt = 1
        else:   # 높이 차이가 2이상 나면 false 리턴.
            return False
    return True

T = int(input())
for tc in range(1, T+1):
    N, L = map(int, input().split())
    arr = [[int(a) for a in input().split()]for _ in range(N)]
    ans = 0

    for _ in range(2):
        for lst in arr:
            v = [0] * len(lst)  # 경사면을 겹쳐서 놓지 않도록 확인하기 위한 리스트
            if check(lst, v) and check(lst[::-1], v[::-1]): # 왕복으로 체크해서 둘다 가능하면
                ans += 1

        arr = list(map(list, zip(*arr)))    # 열을 행으로 전치 행렬

    print(f'#{tc} {ans}')

- 접근방법: 각 행을 처음엔 오른쪽으로 순회하면서 오르막길이 생기면 경사로를 설치할 수 있는지 확인 -> 다시 행의 끝에서 역방향으로 오르막길이 생기면  경사로를 설치할 수 있는지 확인. 열의 경우 전치행렬을 통해 행으로 변경한 후 같은 연산 반복.

## 등산로 조성

In [ ]:
def dfs(y, x, current_value, cnt, k_used):
    global max_root

    # 이동할 네 방향 정의 (상, 우, 하, 좌)
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]

    # 더 이상 갈 수 있는 길이 없다고 가정
    found_path = False

    for dy, dx in directions:
        ny, nx = y + dy, x + dx

        if 0 <= ny < N and 0 <= nx < N and not visited[ny][nx]:
            if current_value > arr[ny][nx]:
                # 현재 높이가 다음 위치보다 크면 이동 가능
                visited[ny][nx] = True
                dfs(ny, nx, arr[ny][nx], cnt + 1, k_used)
                visited[ny][nx] = False
                found_path = True
            elif not k_used and current_value > arr[ny][nx] - K:
                # 현재 위치보다 높지만 K만큼 깎아서 이동 가능하면 이동
                visited[ny][nx] = True
                original_value = arr[ny][nx]
                arr[ny][nx] = current_value - 1
                dfs(ny, nx, arr[ny][nx], cnt + 1, True)
                arr[ny][nx] = original_value
                visited[ny][nx] = False
                found_path = True

    # 더 이상 이동할 수 있는 경로가 없다면, max_root 갱신
    if not found_path:
        max_root = max(max_root, cnt)


# 입력 처리 및 문제 해결
T = int(input())
for tc in range(1, T + 1):
    N, K = map(int, input().split())
    arr = [list(map(int, input().split())) for _ in range(N)]

    max_height = max(map(max, arr))
    max_root = 0
    visited = [[False] * N for _ in range(N)]

    for i in range(N):
        for j in range(N):
            if arr[i][j] == max_height:
                visited[i][j] = True
                dfs(i, j, arr[i][j], 1, False)
                visited[i][j] = False

    print(f'#{tc} {max_root}')

- 가장 높은 봉우리의 좌표를 top리스트에 저장하고 그 top리스트의 길이만큼 dfs함수 반복.
- 상하좌우로 이동하면서 현재 높이가 다음 높이보다 높으면 재귀하면서 cnt를 1씩증가
- 다음 높이가 현재 높이보다 높다면 -> 산을 깎은 이력이 있는지, 최대 k만큼 깎으면 현재 높이보다 낮아지는지 여부를 판단하고 다시 재귀하는 방식을 반복
- found_path를 이용해서 경로를 찾았다면 true, 못찾았다면 해당 시점에서 cnt 값과 max_root와 비교하여 max_root갱신.